In [1]:
pip install basketball-reference-scraper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 2.9 MB/s 
     |████████████████████████████████| 227 kB 9.5 MB/s 
     |████████████████████████████████| 11.7 MB 69.9 MB/s 
     |████████████████████████████████| 15.7 MB 45.1 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 509 kB 44.1 MB/s 
     |████████████████████████████████| 6.9 MB 30.2 MB/s 
     |████████████████████████████████| 241 kB 55.8 MB/s 
     |████████████████████████████████| 106 kB 29.7 MB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: pytz
    Found existing installation: pytz 2022.6
    Uninstalling pytz-2022.6:
      Successfully uninstalled pytz-2022.6
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8

In [2]:
from basketball_reference_scraper.shot_charts import get_shot_chart
from basketball_reference_scraper.seasons import get_schedule
import pandas as pd
import time

In [3]:
def schedule(team, years):
  print(f'Team: {team}')
  full_schedule = pd.DataFrame()
  for i in years:
    schedule_i = get_schedule(i, playoffs=False)
    home = schedule_i.loc[schedule_i['HOME'] == team]
    away = schedule_i.loc[schedule_i['VISITOR'] == team]
    frames = [home, away]
    year_team = pd.concat(frames)
    print(f'year - {i}')
    time.sleep(10)
    frames = [full_schedule, year_team]
    full_schedule = pd.concat(frames)
  return full_schedule.sort_values(by=['DATE']).reset_index()

In [4]:
def empty_game_df():
  empty_df = pd.DataFrame(columns=['x', 'y', 'QUARTER', 'TIME_REMAINING', 'PLAYER', 'MAKE_MISS', 'VALUE', 'DISTANCE', 'team'])
  return empty_df

In [5]:
def get_shots(game_df, empty_df, team):
  home = list(game_df['HOME'])
  away = list(game_df['VISITOR'])
  # dates = [date.strftime('%Y-%m-%d') for date in list(game_df['DATE'])]
  dates = game_df['DATE']
  for i in range(len(dates)):
    print('-' * 50)
    print(dates[i])
    print(f'{home[i]} vs. {away[i]}')
    temp_dic = get_shot_chart(dates[i], home[i], away[i])
    print('got game')
    brk_df = temp_dic[team]
    frames = [empty_df, brk_df]
    empty_df =pd.concat(frames)
    time.sleep(10)
  return empty_df
    

In [6]:
def get_player_shots(player, shots_df):
  return shots_df.loc[shots_df['PLAYER'] == player]

In [7]:
def get_player_csv(team, years, player):
  '''
  inputs: 
  1. team - full name of NBA team (str)
  2. years - list of years (list)
  3. player - full name of a player (str)

  output:
  1. player_shots - csv containing the player shots over the years in the list
                    while he was in the team 
  '''
  print('Start Scrapping Schedule...')
  schedule_df = schedule(team, years) # get team games
  empty_df = empty_game_df() # build empty shots df
  print('Start Scarpping Shots...')
  shots_df = get_shots(schedule_df, empty_df, team_dict[team.upper()]) # get full shots of team
  print('Seperate Player...')
  player_shots = get_player_shots(player, shots_df) # seperate the df to get player shots
  return player_shots 

In [8]:
team_dict = {'ATLANTA HAWKS': 'ATL', 'BOSTON CELTICS': 'BOS', 'BROOKLYN NETS': 'BRK', 'CHICAGO BULLS': 'CHI', 'CHARLOTTE HORNETS': 'CHO', 'CLEVELAND CAVALIERS': 'CLE',
             'DALLAS MAVERICKS': 'DAL', 'DENVER NUGGETS': 'DEN', 'DETROIT PISTONS': 'DET', 'GOLDEN STATE WARRIORS': 'GSW', 'HOUSTON ROCKETS': 'HOU',
             'INDIANA PACERS': 'IND', 'LOS ANGELES CLIPPERS': 'LAC', 'LOS ANGELES LAKERS': 'LAL', 'MEMPHIS GRIZZLIES': 'MEM', 'MIAMI HEAT': 'MIA', 
             'MILWAUKEE BUCKS': 'MIL', 'MINNESOTA TIMBERWOLVES': 'MIN', 'NEW ORLEANS PELICANS': 'NOP', 'NEW YORK KNICKS': 'NYK', 'OKLAHOMA CITY THUNDER': 'OKC',
             'ORLANDO MAGIC': 'ORL', 'PHILADELPHIA 76ERS': 'PHI', 'PHOENIX SUNS': 'PHO', 'PORTLAND TRAIL BLAZERS': 'POR', 'SACRAMENTO KINGS': 'SAC',
             'SAN ANTONIO SPURS': 'SAS', 'TORONTO RAPTORS': 'TOR', 'UTAH JAZZ': 'UTA', 'WASHINGTON WIZARDS': 'WAS'}

In [9]:
schedule_df = schedule('Brooklyn Nets', [2020,2021,2022])
schedule_df

Team: Brooklyn Nets
year - 2020
year - 2021
year - 2022


,index,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS
0,8,2019-10-23,Minnesota Timberwolves,127,Brooklyn Nets,126
1,20,2019-10-25,New York Knicks,109,Brooklyn Nets,113
2,38,2019-10-27,Brooklyn Nets,133,Memphis Grizzlies,134
3,60,2019-10-30,Indiana Pacers,118,Brooklyn Nets,108
4,75,2019-11-01,Houston Rockets,116,Brooklyn Nets,123
...,...,...,...,...,...,...
238,1230,2022-04-12,Cleveland Cavaliers,108,Brooklyn Nets,115
239,1241,2022-04-17,Brooklyn Nets,114,Boston Celtics,115
240,1250,2022-04-20,Brooklyn Nets,107,Boston Celtics,114
241,1261,2022-04-23,Boston Celtics,109,Brooklyn Nets,103


In [ ]:
kd_csv = get_player_csv('Brooklyn Nets', [2020,2021,2022], 'Kevin Durant')
kd_csv.head()

In [9]:
kd_csv.to_csv('kd_csv.csv', index=False)